In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
data = pd.concat([train, test])

# data dimensions: # rows, # columns
print train.shape
print test.shape
print data.shape

(1460, 81)
(1459, 80)
(2919, 81)


# Explore variables one at a time

In [3]:
data = data.drop(["Id"], axis = 1)  # drop this variable
data.columns.values  # print all column names

array(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual',
       'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces',
       'Foundation', 'FullBath', 'Functional', 'GarageArea', 'GarageCars',
       'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType',
       'GarageYrBlt', 'GrLivArea', 'HalfBath', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenAbvGr', 'KitchenQual', 'LandContour',
       'LandSlope', 'LotArea', 'LotConfig', 'LotFrontage', 'LotShape',
       'LowQualFinSF', 'MSSubClass', 'MSZoning', 'MasVnrArea',
       'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold', 'Neighborhood',
       'OpenPorchSF', 'OverallCond', 'OverallQual', 'PavedDrive',
       'PoolArea', 'P

## MSSubClass

In [4]:
data["MSSubClass"].isnull().sum()

0

MSSubClass is categorical, though it is coded as numeric

In [5]:
freq = data["MSSubClass"].value_counts()  # frequency table
py.iplot([go.Bar(x = freq.keys(), y = freq.values)])

Combine all the 1 and 1.5 story dwelling types as 1, 2 and 2.5 story types as 2, and the rest as 0.

In [6]:
MSSubClass = data["MSSubClass"].replace([20, 30, 40, 45, 50, 120, 150], 1) \
.replace([60, 70, 75, 160], 2) \
.replace([80, 85, 90, 180, 190], 0)

In [7]:
new = pd.DataFrame({"MSSubClass": MSSubClass})  # create new dataframe
print new["MSSubClass"].value_counts()
new.head()

1    1712
2     854
0     353
Name: MSSubClass, dtype: int64


,MSSubClass
0,2
1,1
2,2
3,2
4,2


## MSZoning

In [8]:
data["MSZoning"].isnull().sum()

4

In [9]:
data["MSZoning"].value_counts()

RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: MSZoning, dtype: int64

Residential zone?: Y (1) or N (0)

In [10]:
new["MSZoning"] = data["MSZoning"].map(lambda x: 1 if (x == "RL") or (x == "RM") else 0)
new["MSZoning"].value_counts()

1    2725
0     194
Name: MSZoning, dtype: int64

## LotFrontage

In [11]:
data["LotFrontage"].describe()

count    2433.000000
mean       69.305795
std        23.344905
min        21.000000
25%        59.000000
50%        68.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [12]:
new["LotFrontage"] = data["LotFrontage"].fillna(data["LotFrontage"].mean())
new["LotFrontage"].describe()

count    2919.000000
mean       69.305795
std        21.312345
min        21.000000
25%        60.000000
50%        69.305795
75%        78.000000
max       313.000000
Name: LotFrontage, dtype: float64

## LotArea

In [13]:
data["LotArea"].describe()

count      2919.000000
mean      10168.114080
std        7886.996359
min        1300.000000
25%        7478.000000
50%        9453.000000
75%       11570.000000
max      215245.000000
Name: LotArea, dtype: float64

In [14]:
new["LogLotArea"] = np.log(data["LotArea"])
new["LogLotArea"].describe()

count    2919.000000
mean        9.094785
std         0.509966
min         7.170120
25%         8.919721
50%         9.154087
75%         9.356171
max        12.279532
Name: LogLotArea, dtype: float64

## Street and PavedDrive

In [15]:
data["Street"].isnull().sum()

0

In [16]:
data["Street"].value_counts()

Pave    2907
Grvl      12
Name: Street, dtype: int64

Paved street?: Y (1) or No (0)

In [17]:
Street = pd.Series(np.where(data["Street"] == "Pave", 1, 0))
Street.value_counts()

1    2907
0      12
dtype: int64

Street has a severe lack of variability, providing little-to-no information in predicting SalePrice. Thus, this variable will not be used in the model.

In [18]:
data["PavedDrive"].isnull().sum()

0

In [19]:
data["PavedDrive"].value_counts()

Y    2641
N     216
P      62
Name: PavedDrive, dtype: int64

Paved driveway?: Y (1) or No (0)

In [20]:
new["PavedDrive"] = np.where(data["PavedDrive"] == "Y", 1, 0)
new["PavedDrive"].value_counts()

1    2641
0     278
Name: PavedDrive, dtype: int64

## Alley

In [21]:
data["Alley"].isnull().sum()

2721

NA means that houses have no alley, which is not the same as a missing value. Alley access?: Y (1) or N (0)

In [22]:
new["Alley"] = np.where(data["Alley"].isnull(), 0, 1)
new["Alley"].value_counts()

0    2721
1     198
Name: Alley, dtype: int64

## LotShape

In [23]:
data["LotShape"].isnull().sum()

0

In [24]:
data["LotShape"].value_counts()

Reg    1859
IR1     968
IR2      76
IR3      16
Name: LotShape, dtype: int64

Regular lot shape?: Y (1) or No (0)

In [25]:
new["LotShape"] = np.where(data["LotShape"] == "Reg", 1, 0)
new["LotShape"].value_counts()

1    1859
0    1060
Name: LotShape, dtype: int64

## LandContour

In [26]:
data["LandContour"].isnull().sum()

0

In [27]:
data["LandContour"].value_counts()

Lvl    2622
HLS     120
Bnk     117
Low      60
Name: LandContour, dtype: int64

Flat lot?: Y (1) or N (0)

In [28]:
new["LandContour"] = np.where(data["LandContour"] == "Lvl", 1, 0)
new["LandContour"].value_counts()

1    2622
0     297
Name: LandContour, dtype: int64

## Utilities

In [29]:
data["Utilities"].value_counts()

AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64

Drop this variable. It has a severe lack of variability!

## LotConfig

In [30]:
data["LotConfig"].isnull().sum()

0

In [31]:
data["LotConfig"].value_counts()

Inside     2133
Corner      511
CulDSac     176
FR2          85
FR3          14
Name: LotConfig, dtype: int64

Inside lot?: Y (1) or No (0)

In [32]:
new["LotConfig"] = np.where(data["LotConfig"] == "Inside", 1, 0)
new["LotConfig"].value_counts()

1    2133
0     786
Name: LotConfig, dtype: int64

## LandSlope

In [33]:
data["LandSlope"].isnull().sum()

0

In [34]:
data["LandSlope"].value_counts()

Gtl    2778
Mod     125
Sev      16
Name: LandSlope, dtype: int64

Gentle slope?: Y (1) or No (0)

In [35]:
new["LandSlope"] = np.where(data["LandSlope"] == "Gtl", 1, 0)
new["LandSlope"].value_counts()

1    2778
0     141
Name: LandSlope, dtype: int64

## Neighborhood

In [36]:
data["Neighborhood"].isnull().sum()

0

Based on cluster analysis done previously, there are four neighborhood clusters.

In [37]:
new["Neighborhood"] = data["Neighborhood"] \
.replace(["Blmngtn", "Blueste", "BrDale", "BrkSide", "ClearCr", "Crawfor", "Edwards", "Gilbert"], "CollgCr") \
.replace(["IDOTRR", "MeadowV"], "Mitchel") \
.replace(["NPkVill", "NWAmes", "NoRidge"], "NAmes") \
.replace(["NridgHt", "SWISU", "Sawyer", "SawyerW", "Somerst", "StoneBr", "Timber", "Veenker"], "OldTown")

## Condition1 and Condition2

In [38]:
data["Condition1"].value_counts()

Norm      2511
Feedr      164
Artery      92
RRAn        50
PosN        39
RRAe        28
PosA        20
RRNn         9
RRNe         6
Name: Condition1, dtype: int64

In [39]:
data["Condition2"].value_counts()

Norm      2889
Feedr       13
Artery       5
PosA         4
PosN         4
RRNn         2
RRAn         1
RRAe         1
Name: Condition2, dtype: int64

Most houses have normal condition. If either conditions are normal, record the overall condition as normal. 

In [40]:
Condition = np.logical_or(data["Condition1"] == "Norm", data["Condition2"] == "Norm")
Condition.value_counts()

True     2889
False      30
Name: Condition1, dtype: int64

Drop this variable since it lacks variability.

## BldgType

In [41]:
data["BldgType"].isnull().sum()

0

In [42]:
data["BldgType"].value_counts()

1Fam      2425
TwnhsE     227
Duplex     109
Twnhs       96
2fmCon      62
Name: BldgType, dtype: int64

Single-family detached?: Y (1) or N (0)

In [43]:
new["BldgType"] = np.where(data["BldgType"] == "1Fam", 1, 0)
new["BldgType"].value_counts()

1    2425
0     494
Name: BldgType, dtype: int64

## HouseStyle

In [44]:
data["HouseStyle"].isnull().sum()

0

In [45]:
data["HouseStyle"].value_counts()

1Story    1471
2Story     872
1.5Fin     314
SLvl       128
SFoyer      83
2.5Unf      24
1.5Unf      19
2.5Fin       8
Name: HouseStyle, dtype: int64

Combine 1-1.5 story as 1, 2-2.5 story as 2, and the rest as 0.

In [46]:
new["HouseStyle"] = data["HouseStyle"].replace(["1Story", "1.5Fin", "1.5Unf"], 1) \
.replace(["2Story", "2.5Fin", "2.5Unf"], 2) \
.replace(["SFoyer", "SLvl"], 0)
new["HouseStyle"].value_counts()

1    1804
2     904
0     211
Name: HouseStyle, dtype: int64

## OverallQual

In [47]:
data["OverallQual"].isnull().sum()

0

In [48]:
freq = data["OverallQual"].value_counts()
py.iplot([go.Bar(x = freq.keys(), y = freq.values)])

Combine very poor/poor/fair/less than average  as -1, average/above average/good as 0, very good/excellent/very excellent as 1.

In [49]:
new["OverallQual"] = data["OverallQual"].map(lambda x: -1 if x < 4 else 0 if x < 8 else 1)
new["OverallQual"].value_counts()

 0    2382
 1     480
-1      57
Name: OverallQual, dtype: int64

## OverallCond

In [50]:
data["OverallCond"].isnull().sum()

0

In [51]:
freq = data["OverallCond"].value_counts()
py.iplot([go.Bar(x = freq.keys(), y = freq.values)])

Above average condition?: Y (1) or No (0) 

In [52]:
new["OverallCond"] = np.where(data["OverallCond"] > 4, 1, 0)
new["OverallCond"].value_counts()

1    2751
0     168
Name: OverallCond, dtype: int64

## YearBuilt 

In [53]:
data["YearBuilt"].describe()

count    2919.000000
mean     1971.312778
std        30.291442
min      1872.000000
25%      1953.500000
50%      1973.000000
75%      2001.000000
max      2010.000000
Name: YearBuilt, dtype: float64

Years usually needs to be binned.

In [54]:
new["YearBuilt"] = pd.qcut(data["YearBuilt"], q = 4, labels = ["ancient", "older", "newer", "modern"])
pd.concat((new["YearBuilt"], data["YearBuilt"]), axis = 1).head()

,YearBuilt,YearBuilt
0,modern,2003
1,newer,1976
2,newer,2001
3,ancient,1915
4,newer,2000


In [55]:
new["YearBuilt"].value_counts()

newer      748
older      741
ancient    730
modern     700
Name: YearBuilt, dtype: int64

## YearRemodAdd

In [56]:
data["YearRemodAdd"].describe()

count    2919.000000
mean     1984.264474
std        20.894344
min      1950.000000
25%      1965.000000
50%      1993.000000
75%      2004.000000
max      2010.000000
Name: YearRemodAdd, dtype: float64

In [57]:
remodel = np.subtract(data["YearRemodAdd"], data["YearBuilt"])
remodel.describe()

count    2919.000000
mean       12.951696
std        24.065624
min        -1.000000
25%         0.000000
50%         0.000000
75%        18.000000
max       127.000000
Name: YearRemodAdd, dtype: float64

In [58]:
new["YearRemodAdd"] = pd.Series(remodel.map(lambda x: "Never" if x <= 0 else "recent" if x <= 10 else "long ago"))
new["YearRemodAdd"].value_counts()

Never       1561
long ago     807
recent       551
Name: YearRemodAdd, dtype: int64

## RoofStyle

In [59]:
data["RoofStyle"].isnull().sum()

0

In [60]:
data["RoofStyle"].value_counts()

Gable      2310
Hip         551
Gambrel      22
Flat         20
Mansard      11
Shed          5
Name: RoofStyle, dtype: int64

Gable roof?: Y (1) or No (0)

In [61]:
new["RoofStyle"] = np.where(data["RoofStyle"] == "Gable", 1, 0)
new["RoofStyle"].value_counts()

1    2310
0     609
Name: RoofStyle, dtype: int64

## RoofMatl

In [62]:
data["RoofMatl"].isnull().sum()

0

In [63]:
data["RoofMatl"].value_counts()

CompShg    2876
Tar&Grv      23
WdShake       9
WdShngl       7
Membran       1
ClyTile       1
Metal         1
Roll          1
Name: RoofMatl, dtype: int64

Standard shingle?: Y (1) or N (0)

In [64]:
RoofMatl = pd.Series(np.where(data["RoofMatl"] == "CompShg", 1, 0))
RoofMatl.value_counts()

1    2876
0      43
dtype: int64

Drop this variable since it heavily lacks variability.

## Exterior1st and Exterior2nd

In [65]:
print data["Exterior1st"].isnull().sum()
print data["Exterior2nd"].isnull().sum()

1
1


In [66]:
data["Exterior1st"].value_counts()

VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
AsphShn       2
Stone         2
CBlock        2
ImStucc       1
Name: Exterior1st, dtype: int64

In [67]:
Exterior1st = data["Exterior1st"].fillna("Other") \
.replace(["BrkFace", "WdShing", "AsbShng", "Stucco", "BrkComm", "AsphShn", "Stone", "CBlock", "ImStucc", "Other"], "Other")
Exterior1st.value_counts()

VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Other       244
Plywood     221
CemntBd     126
Name: Exterior1st, dtype: int64

In [68]:
data["Exterior2nd"].value_counts()

VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
BrkFace      47
Stucco       47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: Exterior2nd, dtype: int64

In [69]:
Exterior2nd = data["Exterior2nd"].fillna("Other") \
.replace(["Wd Shng", "BrkFace", "Stucco", "AsbShng", "Brk Cmn", "ImStucc", "Stone", "AsphShn", "CBlock", "Other"], "Other")
Exterior2nd.value_counts()

VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
Other       265
CmentBd     126
Name: Exterior2nd, dtype: int64

In [70]:
np.equal(Exterior1st, Exterior2nd).value_counts()  # check if both columns are same

True     2544
False     375
Name: Exterior1st, dtype: int64

Since both variables roughly have the same distribution and most houses have only one exterior material, keep only one of them in the model.

In [71]:
new["Exterior"] = Exterior1st
new["Exterior"].value_counts()

VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Other       244
Plywood     221
CemntBd     126
Name: Exterior, dtype: int64

## MasVnrType and MasVnrArea

In [72]:
data["MasVnrArea"].describe()

count    2896.000000
mean      102.201312
std       179.334253
min         0.000000
25%         0.000000
50%         0.000000
75%       164.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

In [73]:
data["MasVnrArea"] = data["MasVnrArea"].fillna(0)
np.sum(data["MasVnrArea"] == 0)

1761

Since most houses don't have masonry veneer walls, the area is mostly 0.

In [74]:
data["MasVnrType"].isnull().sum()

24

In [75]:
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["MasVnrType"].value_counts()

None       1766
BrkFace     879
Stone       249
BrkCmn       25
Name: MasVnrType, dtype: int64

It makes more sense to simply record if a house has masonry veneer rather than worrying about unnecessary details, such as its area and type.

In [76]:
new["MasVnr"] = np.where(data["MasVnrArea"] == 0, 0, 1)
new["MasVnr"].value_counts()

0    1761
1    1158
Name: MasVnr, dtype: int64

## ExterQual and ExterCond

In [77]:
print data["ExterQual"].isnull().sum()
print data["ExterCond"].isnull().sum()

0
0


Compare the original quality of the material to its current condition.

In [78]:
original = data["ExterQual"].map({"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1})
current = data["ExterCond"].map({"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1})
original.value_counts()

3    1798
4     979
5     107
2      35
Name: ExterQual, dtype: int64

In [79]:
current.value_counts()

3    2538
4     299
2      67
5      12
1       3
Name: ExterCond, dtype: int64

Get the change in condition over time.

In [80]:
cond = np.subtract(current, original)
cond.value_counts()

 0    1620
-1     951
 1     237
-2     104
 2       7
Name: ExterCond, dtype: int64

In [81]:
new["ExterCond"] = pd.Series(cond.map(lambda x: "depreciated" if x < 0 else "improved" if x > 0 else "no change"))
new["ExterCond"].value_counts()

no change      1620
depreciated    1055
improved        244
Name: ExterCond, dtype: int64

## Foundation

In [82]:
data["Foundation"].isnull().sum()

0

In [83]:
data["Foundation"].value_counts()

PConc     1308
CBlock    1235
BrkTil     311
Slab        49
Stone       11
Wood         5
Name: Foundation, dtype: int64

In [84]:
new["Foundation"] = data["Foundation"].replace(["BrkTil", "Slab", "Stone", "Wood"], "Other")
new["Foundation"].value_counts()

PConc     1308
CBlock    1235
Other      376
Name: Foundation, dtype: int64

## BsmtQual

This variable records the basement height.

In [85]:
data["BsmtQual"].isnull().sum()

81

In [86]:
data["BsmtQual"].value_counts()

TA    1283
Gd    1209
Ex     258
Fa      88
Name: BsmtQual, dtype: int64

NA means no basement, and not missing.

In [87]:
new["BsmtQual"] = data["BsmtQual"].fillna("None").map({"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 1, "None": 0})
new["BsmtQual"].value_counts()

2    1283
3    1209
4     258
1      88
0      81
Name: BsmtQual, dtype: int64

## BsmtCond

In [88]:
data["BsmtCond"].isnull().sum()

82

In [89]:
data["BsmtCond"].value_counts()

TA    2606
Gd     122
Fa     104
Po       5
Name: BsmtCond, dtype: int64

NA means no basement, and not missing.

In [90]:
new["BsmtCond"] = data["BsmtCond"].fillna("None").map({"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 1, "None": 0})
new["BsmtCond"].value_counts()

2    2606
3     122
1     109
0      82
Name: BsmtCond, dtype: int64

Atleast average basement condition?: Y (1) or No (0)

In [91]:
new["BsmtCond"] = np.where(new["BsmtCond"] > 1, 1, 0)
new["BsmtCond"].value_counts()

1    2728
0     191
Name: BsmtCond, dtype: int64

## BsmtExposure

In [92]:
data["BsmtExposure"].isnull().sum()

82

NA means no basement, and not missing.

In [93]:
data["BsmtExposure"].value_counts()

No    1904
Av     418
Gd     276
Mn     239
Name: BsmtExposure, dtype: int64

Has a walkout or garden level walls?: Y (1) or No (0)

In [94]:
new["BsmtExposure"] = data["BsmtExposure"].fillna("None").map({"Gd": 1, "Av": 1, "Mn": 1, "No": 0, "None": 0})
new["BsmtExposure"].value_counts()

0    1986
1     933
Name: BsmtExposure, dtype: int64

## BsmtFinType1 and BsmtFinType2

In [95]:
print data["BsmtFinType1"].isnull().sum()
print data["BsmtFinType2"].isnull().sum()

79
80


NA means no basement, and not missing.

In [96]:
data["BsmtFinType1"].value_counts()

Unf    851
GLQ    849
ALQ    429
Rec    288
BLQ    269
LwQ    154
Name: BsmtFinType1, dtype: int64

In [97]:
data["BsmtFinType2"].value_counts()

Unf    2493
Rec     105
LwQ      87
BLQ      68
ALQ      52
GLQ      34
Name: BsmtFinType2, dtype: int64

In [98]:
type1 = data["BsmtFinType1"].fillna("None").map({"GLQ": 3, "ALQ": 2, "Rec": 2, "BLQ": 1, "LwQ": 1, "Unf": 0, "None": 0})
type2 = data["BsmtFinType2"].fillna("None").map({"GLQ": 3, "ALQ": 2, "Rec": 2, "BLQ": 1, "LwQ": 1, "Unf": 0, "None": 0})
type1.value_counts()

0    930
3    849
2    717
1    423
Name: BsmtFinType1, dtype: int64

In [99]:
type2.value_counts()

0    2573
2     157
1     155
3      34
Name: BsmtFinType2, dtype: int64

In [100]:
np.equal(type1, type2).value_counts()  # most basements have a 2nd rating

False    1899
True     1020
Name: BsmtFinType1, dtype: int64

Most basements can be used as a living quarter, however that space is unfinished. Hence, it makes more sense here to record if the basement is unfinished and use TotalBsmtSF to provide the total basement area.

In [101]:
new["BsmtFinType"] = pd.Series(np.logical_or(type1 == 0.0, type2 == 0.0))
new["BsmtFinType"] = np.where(new["BsmtFinType"] == True, 1, 0)
new["BsmtFinType"].value_counts()

1    2573
0     346
Name: BsmtFinType, dtype: int64

## BsmtFinSF1, BsmtFinSF2, BsmtUnfSF

In [102]:
data["BsmtFinSF1"].describe()

count    2918.000000
mean      441.423235
std       455.610826
min         0.000000
25%         0.000000
50%       368.500000
75%       733.000000
max      5644.000000
Name: BsmtFinSF1, dtype: float64

In [103]:
data["BsmtFinSF2"].describe()

count    2918.000000
mean       49.582248
std       169.205611
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      1526.000000
Name: BsmtFinSF2, dtype: float64

In [104]:
print (data["BsmtFinSF1"] == 0).sum()
print (data["BsmtFinSF2"] == 0).sum()

929
2571


Since most basements are unfinished, there are just as many zeroes as in type1 and type2.

In [105]:
BsmtFinSF = np.logical_or(data["BsmtFinSF1"] == 0, data["BsmtFinSF2"] == 0)
(BsmtFinSF == True).sum()

2571

Drop this variable since it is redundant with BsmtFinType.

In [106]:
data["BsmtUnfSF"].describe()

count    2918.000000
mean      560.772104
std       439.543659
min         0.000000
25%       220.000000
50%       467.000000
75%       805.500000
max      2336.000000
Name: BsmtUnfSF, dtype: float64

In [107]:
(data["BsmtUnfSF"] != 0).sum()

2678

There are many nonzero values since most basements are unfinished. Drop this variable since it is redundant with BsmtFinType.

## TotalBsmtSF

In [108]:
data["TotalBsmtSF"].describe()

count    2918.000000
mean     1051.777587
std       440.766258
min         0.000000
25%       793.000000
50%       989.500000
75%      1302.000000
max      6110.000000
Name: TotalBsmtSF, dtype: float64

In [109]:
data["TotalBsmtSF"] = data["TotalBsmtSF"].fillna(0)
new["LogTotalBsmtSF"] = np.log(data["TotalBsmtSF"] + 1)
new["LogTotalBsmtSF"].describe()

count    2919.000000
mean        6.730833
std         1.182460
min         0.000000
25%         6.677083
50%         6.897705
75%         7.172425
max         8.717846
Name: LogTotalBsmtSF, dtype: float64

## Heating

In [110]:
data["Heating"].value_counts()

GasA     2874
GasW       27
Grav        9
Wall        6
OthW        2
Floor       1
Name: Heating, dtype: int64

Has gas air furnace?: Y (1) or No (0).

In [111]:
Heating = pd.Series(np.where(data["Heating"] == "GasA", 1, 0))
Heating.value_counts()

1    2874
0      45
dtype: int64

Drop this variable since it severely lacks variability.

## HeatingQC

In [112]:
data["HeatingQC"].isnull().sum()

0

In [113]:
data["HeatingQC"].value_counts()

Ex    1493
TA     857
Gd     474
Fa      92
Po       3
Name: HeatingQC, dtype: int64

In [114]:
new["HeatingQC"] = data["HeatingQC"].map({"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 1})
new["HeatingQC"].value_counts()

4    1493
2     857
3     474
1      95
Name: HeatingQC, dtype: int64

## CentralAir

In [115]:
data["CentralAir"].isnull().sum()

0

In [116]:
data["CentralAir"].value_counts()

Y    2723
N     196
Name: CentralAir, dtype: int64

In [117]:
new["CentralAir"] = np.where(data["CentralAir"] == "Y", 1, 0)
new["CentralAir"].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

## Electrical

In [118]:
data["Electrical"].isnull().sum()

1

In [119]:
data["Electrical"].value_counts()

SBrkr    2671
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

Standard breaker?: Y (1) or No (0)

In [120]:
new["Electrical"] = np.where(data["Electrical"] == "SBrkr", 1, 0)
new["Electrical"].value_counts()

1    2671
0     248
Name: Electrical, dtype: int64

## 1stFlrSF

In [121]:
data["1stFlrSF"].describe()

count    2919.000000
mean     1159.581706
std       392.362079
min       334.000000
25%       876.000000
50%      1082.000000
75%      1387.500000
max      5095.000000
Name: 1stFlrSF, dtype: float64

In [122]:
new["Log1stFlrSF"] = np.log(data["1stFlrSF"])
new["Log1stFlrSF"].describe()

count    2919.000000
mean        7.003229
std         0.322956
min         5.811141
25%         6.775366
50%         6.986566
75%         7.235259
max         8.536015
Name: Log1stFlrSF, dtype: float64

## 2ndFlrSF

In [123]:
data["2ndFlrSF"].describe()

count    2919.000000
mean      336.483727
std       428.701456
min         0.000000
25%         0.000000
50%         0.000000
75%       704.000000
max      2065.000000
Name: 2ndFlrSF, dtype: float64

In [124]:
np.sum(data["2ndFlrSF"] == 0)

1668

Has 2nd floor?: Y (1) or No (0).

In [125]:
new["SecondFlr"] = np.where(data["2ndFlrSF"] == 0, 0, 1)
new["SecondFlr"].value_counts()

0    1668
1    1251
Name: SecondFlr, dtype: int64

## GrLivArea

In [126]:
data["GrLivArea"].describe()

count    2919.000000
mean     1500.759849
std       506.051045
min       334.000000
25%      1126.000000
50%      1444.000000
75%      1743.500000
max      5642.000000
Name: GrLivArea, dtype: float64

In [127]:
np.corrcoef(data["GrLivArea"], data["1stFlrSF"])

array([[ 1.        ,  0.56253825],
       [ 0.56253825,  1.        ]])

There is a positive moderate correlation between GrLivArea and 1stFlrSF.

In [128]:
new["LogGrLivArea"] = np.log(data["GrLivArea"])
new["LogGrLivArea"].describe()

count    2919.000000
mean        7.260762
std         0.324991
min         5.811141
25%         7.026427
50%         7.275172
75%         7.463650
max         8.637994
Name: LogGrLivArea, dtype: float64

## TotalArea: NEW feature

Ground living area is calculated by measuring the outside perimeter of the house and includes only finished, habitable, above-grade living space. Finished basements and unfinished attic areas are not included in total gross living area. Let's combine the ground living area with the total basement area to inform homeowners how much total area will be available to them. Note that the living area on the first floor is included in the ground living area calculations, which explains the correlation between the two variables.

In [129]:
TotalArea = pd.Series(np.sum([data["GrLivArea"], data["TotalBsmtSF"]], axis = 0))
TotalArea.describe()

count     2919.000000
mean      2552.177115
std        806.018663
min        334.000000
25%       2004.000000
50%       2453.000000
75%       2996.000000
max      11752.000000
dtype: float64

In [130]:
new["LogTotalArea"] = np.log(TotalArea)
new["LogTotalArea"].describe()

count    2919.000000
mean        7.804788
std         0.313241
min         5.811141
25%         7.607878
50%         7.815207
75%         8.009363
max         9.371779
Name: LogTotalArea, dtype: float64

## LowQualFinSF

In [131]:
data["LowQualFinSF"].describe()

count    2919.000000
mean        4.694416
std        46.396825
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      1064.000000
Name: LowQualFinSF, dtype: float64

In [132]:
np.sum(data["LowQualFinSF"] == 0)

2879

Drop this variable due to lack of data.

## BsmtFullBath, BsmtHalfBath, FullBath, HalfBath

In [133]:
print data["BsmtFullBath"].isnull().sum()
print data["BsmtHalfBath"].isnull().sum()
print data["FullBath"].isnull().sum()
print data["HalfBath"].isnull().sum()

2
2
0
0


In [134]:
data["BsmtFullBath"].value_counts()

0.0    1705
1.0    1172
2.0      38
3.0       2
Name: BsmtFullBath, dtype: int64

In [135]:
data["FullBath"].value_counts()

2    1530
1    1309
3      64
0      12
4       4
Name: FullBath, dtype: int64

Combine all full bathrooms, regardless of whether it is in the basement or not.

In [136]:
data["BsmtFullBath"] = data["BsmtFullBath"].fillna(0)
full = pd.Series(np.sum([data["BsmtFullBath"], data["FullBath"]], axis = 0))
full.value_counts()

2.0    1466
1.0     758
3.0     645
4.0      44
6.0       5
0.0       1
dtype: int64

In [137]:
new["FullBath"] = full.replace([0], 1).replace([3, 4, 6], 3)
new["FullBath"].value_counts()

2.0    1499
1.0     744
3.0     676
Name: FullBath, dtype: int64

In [138]:
data["BsmtHalfBath"].value_counts()

0.0    2742
1.0     171
2.0       4
Name: BsmtHalfBath, dtype: int64

In [139]:
data["HalfBath"].value_counts()

0    1834
1    1060
2      25
Name: HalfBath, dtype: int64

Similarly, combine all half bathrooms.

In [140]:
data["BsmtHalfBath"] = data["BsmtHalfBath"].fillna(0)
half = pd.Series(np.sum([data["BsmtHalfBath"], data["HalfBath"]], axis = 0))
half.value_counts()

0.0    1700
1.0    1154
2.0      61
3.0       3
4.0       1
dtype: int64

Has half bathroom?: Y (1) or No (0)

In [141]:
new["HalfBath"] = half.replace([2, 3, 4], 1)
new["HalfBath"].value_counts()

0.0    1710
1.0    1209
Name: HalfBath, dtype: int64

## Bedroom

In [142]:
data["BedroomAbvGr"].value_counts()

3    1596
2     742
4     400
1     103
5      48
6      21
0       8
8       1
Name: BedroomAbvGr, dtype: int64

In [143]:
data["BedroomAbvGr"].isnull().sum()

0

In [144]:
new["BedroomAbvGr"] = data["BedroomAbvGr"].replace([0], 1).replace([4, 5, 6, 8], 4)
new["BedroomAbvGr"].value_counts()

3    1596
2     742
4     470
1     111
Name: BedroomAbvGr, dtype: int64

## KitchenAbvGr

In [145]:
data["KitchenAbvGr"].value_counts()

1    2785
2     129
0       3
3       2
Name: KitchenAbvGr, dtype: int64

In [146]:
data["KitchenAbvGr"].isnull().sum()

0

Drop this variable due to lack of data. Also, KitchenQual assumes that a kitchen is available.

## KitchenQual

In [147]:
data["KitchenQual"].value_counts()

TA    1492
Gd    1151
Ex     205
Fa      70
Name: KitchenQual, dtype: int64

In [148]:
data["KitchenQual"].isnull().sum()

1

In [149]:
new["KitchenQual"] = data["KitchenQual"].fillna("TA").map({"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 1})
new["KitchenQual"].value_counts()

2    1493
3    1151
4     205
1      70
Name: KitchenQual, dtype: int64

Above average kitchen quality?: Y (1) or No (0)

In [150]:
new["KitchenQual"] = np.where(new["KitchenQual"] > 2, 1, 0)
new["KitchenQual"].value_counts()

0    1563
1    1356
Name: KitchenQual, dtype: int64

## TotRmsAbvGrd

In [151]:
freq = data["TotRmsAbvGrd"].value_counts()
py.iplot([go.Bar(x = freq.keys(), y = freq.values)])

In [152]:
new["TotRmsAbvGrd"] = data["TotRmsAbvGrd"].replace([2, 3], 4).replace([10, 11, 12, 13, 14, 15], 9)
new["TotRmsAbvGrd"].value_counts()

6    844
7    649
5    583
8    347
9    274
4    222
Name: TotRmsAbvGrd, dtype: int64

## Functional

In [153]:
data["Functional"].value_counts()

Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: Functional, dtype: int64

In [154]:
data["Functional"].isnull().sum()

2

Typical home functionality?: Y (1) or No (0)

In [155]:
data["Functional"] = data["Functional"].fillna("Typ")
new["Functional"] = np.where(data["Functional"] == "Typ", 1, 0)
new["Functional"].value_counts()

1    2719
0     200
Name: Functional, dtype: int64

## Fireplaces

In [156]:
data["Fireplaces"].isnull().sum()

0

In [157]:
data["Fireplaces"].value_counts()

0    1420
1    1268
2     219
3      11
4       1
Name: Fireplaces, dtype: int64

Has a fireplace?: Y (1) or No (0)

In [158]:
new["Fireplaces"] = np.where(data["Fireplaces"] > 0, 1, 0)
new["Fireplaces"].value_counts()

1    1499
0    1420
Name: Fireplaces, dtype: int64

## FireplaceQu

In [159]:
data["FireplaceQu"].isnull().sum()

1420

In [160]:
data["FireplaceQu"].value_counts()

Gd    744
TA    592
Fa     74
Po     46
Ex     43
Name: FireplaceQu, dtype: int64

NA means no fireplace, and not missing.

In [161]:
new["FireplaceQu"] = data["FireplaceQu"].fillna("None").map({"Ex": 3, "Gd": 3, "TA": 2, "Fa": 1, "Po": 1, "None": 0})
new["FireplaceQu"].value_counts()

0    1420
3     787
2     592
1     120
Name: FireplaceQu, dtype: int64

## GarageType

In [162]:
data["GarageType"].isnull().sum()

157

NA means No Garage, and doesn't mean missing.

In [163]:
data["GarageType"] = data["GarageType"].fillna("None")
data["GarageType"].value_counts()

Attchd     1723
Detchd      779
BuiltIn     186
None        157
Basment      36
2Types       23
CarPort      15
Name: GarageType, dtype: int64

Attached garage?: Y (1) or No (0)

In [164]:
new["GarageType"] = np.where(data["GarageType"] == "Attchd", 1, 0)
new["GarageType"].value_counts()

1    1723
0    1196
Name: GarageType, dtype: int64

## GarageYrBlt

In [165]:
data["GarageYrBlt"].describe()

count    2760.000000
mean     1978.113406
std        25.574285
min      1895.000000
25%      1960.000000
50%      1979.000000
75%      2002.000000
max      2207.000000
Name: GarageYrBlt, dtype: float64

In [166]:
data["GarageYrBlt"].isnull().sum()

159

Since 159 houses don't have garages, replace NA with 0, including the max year!

In [167]:
data["GarageYrBlt"] = data["GarageYrBlt"].fillna(0).replace([data["GarageYrBlt"].max()], 0)
data["GarageYrBlt"].describe()

count    2919.000000
mean     1869.608085
std       450.973653
min         0.000000
25%      1957.000000
50%      1977.000000
75%      2001.000000
max      2010.000000
Name: GarageYrBlt, dtype: float64

In [168]:
new["GarageYrBlt"] = pd.qcut(data["YearBuilt"], q = 4, labels = ["ancient", "older", "newer", "modern"])
new["GarageYrBlt"].value_counts()

newer      748
older      741
ancient    730
modern     700
Name: GarageYrBlt, dtype: int64

## GarageFinish

In [169]:
data["GarageFinish"].isnull().sum()

159

NA means no garage, and not missing.

In [170]:
data["GarageFinish"] = data["GarageFinish"].fillna("None")
data["GarageFinish"].value_counts()

Unf     1230
RFn      811
Fin      719
None     159
Name: GarageFinish, dtype: int64

In [171]:
new["GarageFinish"] = data["GarageFinish"]
new["GarageFinish"].value_counts()

Unf     1230
RFn      811
Fin      719
None     159
Name: GarageFinish, dtype: int64

## GarageCars and GarageArea

In [172]:
data["GarageCars"].isnull().sum()

1

In [173]:
data["GarageCars"].value_counts()

2.0    1594
1.0     776
3.0     374
0.0     157
4.0      16
5.0       1
Name: GarageCars, dtype: int64

In [174]:
data["GarageArea"].describe()

count    2918.000000
mean      472.874572
std       215.394815
min         0.000000
25%       320.000000
50%       480.000000
75%       576.000000
max      1488.000000
Name: GarageArea, dtype: float64

In [175]:
data["GarageCars"] = data["GarageCars"].fillna(2)
data["GarageArea"] = data["GarageArea"].fillna(data["GarageArea"].mean())
py.iplot([go.Scatter(x = data["GarageArea"], y = data["GarageCars"], mode = "markers")])

In [176]:
np.corrcoef(data["GarageCars"], data["GarageArea"])

array([[ 1.       ,  0.8896861],
       [ 0.8896861,  1.       ]])

It is clear from the graph that there is a relationship between GarageCars and GarageArea. In fact, both variables have a strong positive correlation of 0.89. Garages that can fit 1-3 cars form the three biggest clusters in the graph. We choose to merge garages that can fit more than 3 cars with the garages that can fit exactly three cars since they share same area square footage range and that data doesn't have much density. Also, in order to reduce redundancy, we keep only one of the two variables. We picked GarageCars since most people understand how many cars they can fit in their garage as opposed to its area. In fact, the number of cars that a garage can fit can be viewed as binning classes for the garage area.

In [177]:
new["GarageCars"] = data["GarageCars"].replace([4, 5], 3)
new["GarageCars"].value_counts()

2.0    1595
1.0     776
3.0     391
0.0     157
Name: GarageCars, dtype: int64

## GarageQual and GarageCond

In [178]:
data["GarageQual"].isnull().sum()

159

NA means no garage, and not missing.

In [179]:
data["GarageQual"] = data["GarageQual"].fillna("None")
data["GarageQual"].value_counts()

TA      2604
None     159
Fa       124
Gd        24
Po         5
Ex         3
Name: GarageQual, dtype: int64

In [180]:
data["GarageCond"].isnull().sum()

159

In [181]:
data["GarageCond"] = data["GarageCond"].fillna("None")
data["GarageCond"].value_counts()

TA      2654
None     159
Fa        74
Gd        15
Po        14
Ex         3
Name: GarageCond, dtype: int64

Let's compare the original garage quality (GarageQual) to the current garage condition (GarageCond).

In [182]:
original = data["GarageQual"].map({"Ex": 4, "Gd": 4, "TA": 2, "Fa": 1, "Po": 1, "None": 0})
current = data["GarageCond"].map({"Ex": 4, "Gd": 4, "TA": 2, "Fa": 1, "Po": 1, "None": 0})
cond = np.subtract(current, original)
cond.value_counts()

 0    2779
 1      77
-1      36
-2      18
 2       9
Name: GarageCond, dtype: int64

In [183]:
new["GarageRemod"] = pd.Series(cond.map(lambda x: "depreciated" if x < 0 else "improved" if x > 0 else "no change"))
new["GarageRemod"].value_counts()

no change      2779
improved         86
depreciated      54
Name: GarageRemod, dtype: int64

Was garage remodeled?: Y (1) or No (0).

In [184]:
new["GarageRemod"] = np.where(new["GarageRemod"] == "no change", 1, 0)
new["GarageRemod"].value_counts()

1    2779
0     140
Name: GarageRemod, dtype: int64

## WoodDeckSF

In [185]:
data["WoodDeckSF"].describe()

count    2919.000000
mean       93.709832
std       126.526589
min         0.000000
25%         0.000000
50%         0.000000
75%       168.000000
max      1424.000000
Name: WoodDeckSF, dtype: float64

In [186]:
np.sum(data["WoodDeckSF"] == 0)

1523

Has a wood deck?: Y(1) or No (0)

In [187]:
new["WoodDeck"] = np.where(data["WoodDeckSF"] == 0, 0, 1)
new["WoodDeck"].value_counts()

0    1523
1    1396
Name: WoodDeck, dtype: int64

## OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch

In [188]:
data["OpenPorchSF"].describe()

count    2919.000000
mean       47.486811
std        67.575493
min         0.000000
25%         0.000000
50%        26.000000
75%        70.000000
max       742.000000
Name: OpenPorchSF, dtype: float64

In [189]:
np.sum(data["OpenPorchSF"] == 0)

1298

In [190]:
data["EnclosedPorch"].describe()

count    2919.000000
mean       23.098321
std        64.244246
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      1012.000000
Name: EnclosedPorch, dtype: float64

In [191]:
np.sum(data["EnclosedPorch"] == 0)

2460

In [192]:
data["3SsnPorch"].describe()

count    2919.000000
mean        2.602261
std        25.188169
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       508.000000
Name: 3SsnPorch, dtype: float64

In [193]:
np.sum(data["3SsnPorch"] == 0)

2882

In [194]:
data["ScreenPorch"].describe()

count    2919.000000
mean       16.062350
std        56.184365
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       576.000000
Name: ScreenPorch, dtype: float64

In [195]:
np.sum(data["ScreenPorch"] == 0)

2663

In [196]:
new["TotalPorchSF"] = np.sum([data["OpenPorchSF"], data["EnclosedPorch"], data["3SsnPorch"], data["ScreenPorch"]], axis = 0)
new["TotalPorchSF"].describe()

count    2919.000000
mean       89.249743
std       107.902805
min         0.000000
25%         0.000000
50%        50.000000
75%       136.000000
max      1207.000000
Name: TotalPorchSF, dtype: float64

In [197]:
np.sum(new["TotalPorchSF"] == 0)

873

## PoolArea and PoolQC

In [198]:
data["PoolArea"].describe()

count    2919.000000
mean        2.251799
std        35.663946
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       800.000000
Name: PoolArea, dtype: float64

In [199]:
np.sum(data["PoolArea"] == 0)

2906

An overwhelming majority of the houses don't have a pool! Drop both PoolArea and PoolQC.

In [200]:
data["PoolQC"].value_counts()

Ex    4
Gd    4
Fa    2
Name: PoolQC, dtype: int64

## Fence

In [201]:
data["Fence"].isnull().sum()

2348

NA means no fence, and not missing. Has Fence?: Y (1) or No (0)

In [202]:
new["Fence"] = np.where(data["Fence"].isnull(), 0 , 1)
new["Fence"].value_counts()

0    2348
1     571
Name: Fence, dtype: int64

## MiscFeature and MiscVal

In [203]:
data["MiscFeature"].isnull().sum()

2814

NA means no miscellaneous features, and not missing. Since vast majority of the houses don't have miscellaneous features, there is no need to investigate its dollar value. Drop both variables.

In [204]:
data["MiscVal"].describe()

count     2919.000000
mean        50.825968
std        567.402211
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      17000.000000
Name: MiscVal, dtype: float64

In [205]:
np.sum(data["MiscVal"] == 0)

2816

## MoSold

In [206]:
data["MoSold"].isnull().sum()

0

In [207]:
freq = data["MoSold"].value_counts()
py.iplot([go.Bar(x = freq.keys(), y = freq.values)])

In [208]:
new["MoSold"] = data["MoSold"] \
.map({12: "winter", 1: "winter", 2: "winter",
    3: "spring", 4: "spring", 5: "spring",
    6: "summer", 7: "summer", 8: "summer",
    9: "fall", 10: "fall", 11: "fall"})
new["MoSold"].value_counts()

summer    1182
spring     905
fall       473
winter     359
Name: MoSold, dtype: int64

## YrSold

In [209]:
data["YrSold"].value_counts()

2007    692
2009    647
2008    622
2006    619
2010    339
Name: YrSold, dtype: int64

In [210]:
data["YrSold"].describe()

count    2919.000000
mean     2007.792737
std         1.314964
min      2006.000000
25%      2007.000000
50%      2008.000000
75%      2009.000000
max      2010.000000
Name: YrSold, dtype: float64

In [211]:
new["YrSold"] = data["YrSold"]

## SaleType

In [212]:
data["SaleType"].isnull().sum()

1

In [213]:
data["SaleType"].value_counts()

WD       2525
New       239
COD        87
ConLD      26
CWD        12
ConLI       9
ConLw       8
Oth         7
Con         5
Name: SaleType, dtype: int64

Combine all types of warranty deed and all types of contracts, but keep newly constructed and sold homes separate.

In [214]:
new["SaleType"] = data["SaleType"] \
.replace(["CWD", "VWD"], "WD") \
.replace(["COD", "Con", "ConLw", "ConLI", "ConLD", "Oth"], "Other") \
.fillna("WD")
new["SaleType"].value_counts()

WD       2538
New       239
Other     142
Name: SaleType, dtype: int64

## SaleCondition

In [215]:
data["SaleCondition"].isnull().sum()

0

In [216]:
data["SaleCondition"].value_counts()

Normal     2402
Partial     245
Abnorml     190
Family       46
Alloca       24
AdjLand      12
Name: SaleCondition, dtype: int64

Normal sale?: Y (1) or No (0)

In [217]:
new["SaleCondition"] = np.where(data["SaleCondition"] == "Normal", 1, 0)
new["SaleCondition"].value_counts()

1    2402
0     517
Name: SaleCondition, dtype: int64

# Save cleaned-up data

In [218]:
data["SalePrice"].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [219]:
new[:1460].to_csv("../data/trainEng.csv", index = False)  # extract rows 0 to 1459
  # append SalePrice column to CSV file
tmp = pd.read_csv("../data/trainEng.csv")
tmp["LogSalePrice"] = np.log(train["SalePrice"])
tmp.to_csv("../data/trainEng.csv", index = False)
new[1460:].to_csv("../data/testEng.csv", index = False)  # extract rest of the rows